In [1]:
import random
from copy import deepcopy
from typing import Tuple

import numpy as np
from numpy.core.umath_tests import inner1d
import torch
import torch.nn as nn
import os

<ipython-input-1-37515f19f01a>:6: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Beam:

In [3]:
class Beam(object):
    def __init__(self, coords=None, intensity=None,
                 dv: np.array(3, float) = None):
        self.coords = coords
        self.intensity = intensity
        self.dv = dv

### RectPlane:

In [4]:
class RectPlane(object):
    def __init__(self, length: float, width: float, thickness: float = 0.0):
        self.length = length
        self.width = width
        self.thickness = thickness
        self.edges = np.stack([[length / 2, width / 2, thickness],
                               [-length / 2, width / 2, thickness],
                               [-length / 2, -width / 2, thickness],
                               [length / 2, -width / 2, thickness]])
        self.parametric = self._set_parametric()
        self.beam_profile = Beam()

    def _set_parametric(self):
        return np.stack([self.edges[0],
                         self.edges[1] - self.edges[0],
                         self.edges[3] - self.edges[0]])

    def _actualize_parametric(self):
        self.parametric = self._set_parametric()

    def _rotate(self, function, angle):
        angle = angle * (np.pi / 180)
        return function(angle).dot(self.edges.T).transpose(1, 0)

    def rotate_x(self, angle):
        self.edges = self._rotate(self.rx_mtx, angle)
        self._actualize_parametric()

    def rotate_y(self, angle):
        self.edges = self._rotate(self.ry_mtx, angle)
        self._actualize_parametric()

    def rotate_z(self, angle):
        self.edges = self._rotate(self.rz_mtx, angle)
        self._actualize_parametric()

    def _shift(self, distance, axis):
        self.edges = self.edges.T
        self.edges[axis] += distance
        self.edges = self.edges.T
        self._actualize_parametric()

    def shift_x(self, distance):
        self._shift(distance, 0)

    def shift_y(self, distance):
        self._shift(distance, 1)

    def shift_z(self, distance):
        self._shift(distance, 2)

    @staticmethod
    def rx_mtx(angle):
        cs, sn = np.cos(angle), np.sin(angle)
        return np.array([[1, 0, 0], [0, cs, -sn], [0, sn, cs]])

    @staticmethod
    def ry_mtx(angle):
        cs, sn = np.cos(angle), np.sin(angle)
        return np.array([[cs, 0, sn], [0, 1, 0], [-sn, 0, cs]])

    @staticmethod
    def rz_mtx(angle):
        cs, sn = np.cos(angle), np.sin(angle)
        return np.array([[cs, -sn, 0], [sn, cs, 0], [0, 0, 1]])

    @staticmethod
    def plane_line_intersect(plane: np.array([float]), coords: np.array([float]), dv: np.array([float])):
        """

        :param plane: [point on plane, normal vector]
        :param coords: [[x1, y1, z1], [x2, y2, z2], ... , [xn, yn, zn]]
        :param dv: direction vector of incoming ray [x, y, z]
        :return: intersection point of every incoming ray with the plane
        """
        w = coords - plane[0]
        fac = w.dot(-np.array(plane[1])) / (dv.dot(plane[1]))
        fac = np.reshape(fac, (-1, 1))
        u = dv * fac
        result = coords + u
        if len(coords.T[0].shape) > 0:
            result[np.where(fac.reshape(coords.T[0].shape[0]) < 0)] = np.array([-10000, -10000, -10000])
        return result

    def reflection(self, line: np.array([float])) -> np.array([float]):
        """

        :param line: first entry: point on line, second entry: direction vector
        :return: reflected direction vector
        """
        plane = np.stack([self.parametric[0], np.cross(self.parametric[1], self.parametric[2])])
        mirror_point_plane = self.plane_line_intersect(plane, line[0], plane[1])
        mirror_point = line[0] + 2 * (mirror_point_plane - line[0])
        reflection_point_plane = self.plane_line_intersect(plane, line[0], line[1])
        reflection_point = mirror_point + 2 * (reflection_point_plane - mirror_point)
        return (reflection_point - reflection_point_plane)[0]

    def get_intensity(self, incoming, apply_noise: bool = False):
        cross_product = np.cross(self.parametric[1], self.parametric[2])
        norm_cross_product = cross_product / np.sqrt(np.dot(cross_product, cross_product))
        sample_plane_in_normal = np.stack([self.parametric[0], norm_cross_product])
        intersect = self.plane_line_intersect(sample_plane_in_normal, incoming.beam_profile.coords,
                                              incoming.beam_profile.dv)
        intersect_new_coordinates = intersect - self.parametric[0]
        # Projection 1:
        fac1 = self.parametric[1] / np.dot(self.parametric[1], self.parametric[1])
        ca = (np.dot(intersect_new_coordinates, self.parametric[1]) * fac1.reshape(-1, 1)).T
        cond1 = inner1d(ca, ca) <= inner1d(self.parametric[1], self.parametric[1])
        cond2 = inner1d(ca, self.parametric[1]) >= 0

        # Projection 2:
        fac2 = self.parametric[2] / np.dot(self.parametric[2], self.parametric[2])
        cb = (np.dot(intersect_new_coordinates, self.parametric[2]) * fac2.reshape(-1, 1)).T
        cond3 = inner1d(cb, cb) <= inner1d(self.parametric[2], self.parametric[2])
        cond4 = inner1d(cb, self.parametric[2]) >= 0

        cond_together = cond1 & cond2 & cond3 & cond4
        indices = np.where(cond_together)
        self.beam_profile.coords = intersect[indices]
        self.beam_profile.dv = self.reflection(np.stack([incoming.parametric[0], incoming.beam_profile.dv]))
        if apply_noise:
            noise_level = 0.0001
            self.beam_profile.intensity = np.random.poisson(incoming.beam_profile.intensity[indices] / noise_level) * noise_level
        else:
            self.beam_profile.intensity = incoming.beam_profile.intensity[indices]

### Incoming:

In [5]:
class Incoming(RectPlane):
    def __init__(self, length: float, width: float, shape: Tuple[str, float, float], inc_intensity: float,
                 gauss: bool = True, amp: float = 1.0, sigma: float = 1.0, ray_numb: float = 10000,
                 det_distance: float = 10.0):
        super().__init__(length, width)
        self.center = np.array([det_distance, 0, 0])
        self.rotate_y(90)
        self.shift_x(det_distance)
        self.set_mesh(ray_numb, det_distance)
        self.set_profile(shape, inc_intensity, amp, sigma, gauss)

    def set_mesh(self, ray_numb, det_distance):
        y, z = np.meshgrid(np.linspace(-self.edges[0][1], self.edges[0][1], int(np.sqrt(ray_numb))),
                           np.linspace(-self.edges[0][2], self.edges[0][2], int(np.sqrt(ray_numb))))
        z, y = z.flatten(), y.flatten()
        x = np.ones_like(z) * det_distance
        intensity = np.zeros_like(z)
        coords = np.stack([x, y, z]).T
        dv = np.array([-1, 0, 0])
        self.beam_profile = Beam(coords, intensity, dv)

    def set_profile(self, shape, inc_intensity, amp, sigma, gauss):
        if gauss:
            self.beam_profile.intensity = self.create_gauss_pattern(amp, sigma)
        else:
            self.beam_profile.intensity = np.ones(len(self.beam_profile.coords.T[0]))
        inc_center = self.beam_profile.coords - self.center
        if shape[0] == "round":
            self.beam_profile.intensity[inner1d(inc_center, inc_center) > shape[1]] = 0
        elif shape[0] == "elliptical":
            arr = inc_center / np.array([1, shape[1], shape[2]])
            self.beam_profile.intensity[inner1d(arr, arr) > 1] = 0
        elif shape[0] == "quadratic":
            pass
        else:
            print("Wrong input in shape of incoming beam.")
            print("Possible: round, elliptical, quadratic")
        self.beam_profile.intensity *= inc_intensity / np.sum(self.beam_profile.intensity)

    def create_gauss_pattern(self, amp: float, sigma: float):
        y, z = self.beam_profile.coords.T[1], self.beam_profile.coords.T[2]
        d = np.sqrt(y ** 2 + z ** 2)
        return amp * (np.exp(-(d ** 2 / (2.0 * sigma ** 2)))).flatten()

### Sample:

In [6]:
class Sample(RectPlane):
    def __init__(self, length, width, thickness, det_angle):
        super().__init__(length, width, thickness)
        """self.distance_local_origin = distance_origin_sample + np.array([0, 0, thickness])
        self.edges += self.distance_local_origin
        self.rotate_y(det_angle / 2)
        self._actualize_parametric()"""
        self.edges += np.array([0, 0, thickness])
        self.rotate_y(det_angle / 2)
        self._actualize_parametric()

    def set_position(self, misalignment):
        """

        :param misalignment: Tuple(chi, omega, x, y, z)
        :return: Set the misaligned position of the sample
        """
        self.rotate_x(misalignment[0])
        self.rotate_y(misalignment[1])
        self.shift_x(misalignment[2])
        self.shift_y(misalignment[3])
        self.shift_z(misalignment[4])

### Detector:

In [7]:
class Detector(RectPlane):
    def __init__(self, length: float, width: float, det_angle: float, det_distance: float = 10.0):
        super().__init__(length, width)
        self.rotate_y(90)
        self.shift_x(-det_distance)
        self.rotate_y(det_angle)
        self._actualize_parametric()

    @staticmethod
    def is_point_left(coords, line):
        coords_diff = coords - line[0]
        coords_diff = (coords_diff.T * np.array([[1], [1], [-1]])).T
        dv2 = np.array([line[1][0], line[1][2], line[1][1]])
        return coords_diff.dot(dv2) < 0

    def shadow_scan(self, incoming: Incoming, sample: Sample, apply_noise: bool):
        """
        Unprofessional documentation:
            1. Project sample edges onto y-z-plane (x = 0) -> saved in y_z_edges
            2. Find edge index with highest z value -> highest_edge_idx
            3. Now I want to get the two neighbour edges of considered edge, by looking at the previous and next place in y_z_edges
                - in the cases of A or D this would throw an Error (for example: looking at the previous edge of A would end in looking at position -1, which si impossible)
                - Trick: So adding D at the front of the list and A again at the end, delivers the desired functions -> help_edges
                - Note: highest_edge_idx has to be increased by 1, to get the correct position in help_edges
            4. Take the edge with the highest z value and check if one of the neighbour edges has the same height
                - True: Check if y value of this neighbour is also identical
                    - True: Take line of highest edge and other neighbour
                    - False: Take line of highest edge and this neighbour
                - False: One has to consider two lines, which define the shadowed region
        """
        y_z_edges = sample.edges
        for i in range(len(y_z_edges)):
            y_z_edges[i] = np.array([0, y_z_edges[i][1], y_z_edges[i][2]])
        highest_edge_idx = y_z_edges.T[2].argmax() + 1
        help_edges = np.stack([y_z_edges[3], *y_z_edges, y_z_edges[0]])
        if help_edges[highest_edge_idx][2] == help_edges[highest_edge_idx - 1][2] or help_edges[highest_edge_idx][2] == help_edges[highest_edge_idx + 1][2]:
            if help_edges[highest_edge_idx][2] == help_edges[highest_edge_idx - 1][2] and help_edges[highest_edge_idx][1] != help_edges[highest_edge_idx - 1][1]:
                if help_edges[highest_edge_idx][1] < help_edges[highest_edge_idx - 1][1]:
                    line1 = np.stack([help_edges[highest_edge_idx - 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx - 1]])
                    line2 = deepcopy(line1)
                elif help_edges[highest_edge_idx][1] > help_edges[highest_edge_idx - 1][1]:
                    line1 = np.stack([help_edges[highest_edge_idx], help_edges[highest_edge_idx - 1] - help_edges[highest_edge_idx]])
                    line2 = deepcopy(line1)
                else:
                    line1 = np.stack([help_edges[highest_edge_idx + 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx + 1]])
                    line2 = deepcopy(line1)
            else:
                if help_edges[highest_edge_idx][1] < help_edges[highest_edge_idx + 1][1]:
                    line1 = np.stack([help_edges[highest_edge_idx + 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx + 1]])
                    line2 = deepcopy(line1)
                elif help_edges[highest_edge_idx][1] > help_edges[highest_edge_idx + 1][1]:
                    line1 = np.stack([help_edges[highest_edge_idx], help_edges[highest_edge_idx + 1] - help_edges[highest_edge_idx]])
                    line2 = deepcopy(line1)
                else:
                    line1 = np.stack([help_edges[highest_edge_idx - 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx - 1]])
                    line2 = deepcopy(line1)
        else:
            if help_edges[highest_edge_idx - 1][1] < help_edges[highest_edge_idx + 1][1]:
                line1 = np.stack([help_edges[highest_edge_idx + 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx + 1]])
                line2 = np.stack([help_edges[highest_edge_idx], help_edges[highest_edge_idx - 1] - help_edges[highest_edge_idx]])
            else:
                line1 = np.stack([help_edges[highest_edge_idx - 1], help_edges[highest_edge_idx] - help_edges[highest_edge_idx - 1]])
                line2 = np.stack([help_edges[highest_edge_idx], help_edges[highest_edge_idx + 1] - help_edges[highest_edge_idx]])
        cond1 = self.is_point_left(incoming.beam_profile.coords, line1)
        cond2 = self.is_point_left(incoming.beam_profile.coords, line2)
        cond = cond1 & cond2
        cond_reverse = cond == False
        indices = np.where(cond_reverse)
        det_plane_in_normal = np.stack([self.parametric[0], np.cross(self.parametric[1], self.parametric[2])])
        intersect = self.plane_line_intersect(det_plane_in_normal, incoming.beam_profile.coords,
                                              incoming.beam_profile.dv)
        self.beam_profile.coords = intersect[indices]
        if apply_noise:
            noise_level = 0.0001
            self.beam_profile.intensity = np.random.poisson(incoming.beam_profile.intensity[indices] / noise_level) * noise_level
        else:
            self.beam_profile.intensity = incoming.beam_profile.intensity[indices]

### Scence:

In [8]:
class Scene(object):
    def __init__(self, incoming_props, sample_props, detector_props, misalignment, theta, apply_noise=True, detection_distance=10.0):
        """

        :param incoming_props: Tuple(length, width, shape: Tuple[str, float, float], inc_intensity, max_amplitude, sigma, ray_numbers)
        :param sample_props: Tuple(length, width, thickness)
        :param detector_props: Tuple(length, width)
        :param misalignment: Tuple(chi, omega, x, y, z)
        """
        if torch.is_tensor(misalignment):
            misalignment = misalignment.cpu().detach().numpy()

        if theta == 0:
            self.incoming = Incoming(*incoming_props, det_distance=detection_distance)
            self.sample = Sample(*sample_props, det_angle=theta)
            self.sample.set_position(misalignment)
            self.sample.get_intensity(self.incoming, apply_noise=apply_noise)
            self.detector = Detector(*detector_props, det_angle=theta, det_distance=detection_distance)
            self.detector.shadow_scan(self.incoming, self.sample, apply_noise=apply_noise)
        else:
            self.incoming = Incoming(*incoming_props, det_distance=detection_distance)
            self.sample = Sample(*sample_props, det_angle=theta)
            self.sample.set_position(misalignment)
            self.sample.get_intensity(self.incoming, apply_noise=apply_noise)
            self.detector = Detector(*detector_props, det_angle=theta, det_distance=detection_distance)
            self.detector.get_intensity(self.sample, apply_noise=apply_noise)

### Parameters:

In [9]:
# Properties of Incoming Beam, Sample and Detector:
DETECTION_ANGLE = 20

# IncomingBeam:
inc_length = 2
inc_width = 2
inc_beam_shape = ("round", 0.4, 0.4)
inc_intensity = 1
gauss = True
INC_PROPS = (inc_length, inc_width, inc_beam_shape, inc_intensity, gauss)

# Sample:
sample_length = 8
sample_width = 8
sample_thickness = 0
SAMPLE_PROPS = (sample_length, sample_width, sample_thickness)

# Detector:
detector_length = 2
detector_width = 2
DET_PROPS = (detector_length, detector_width)

"""
MOTOR_RANGES: Defines the maximal motor position of each motor in positive as well as in negative direction.
"""

CHI_RANGE = 1
OMEGA_RANGE = 1
X_RANGE = 0.1
Y_RANGE = 0.1
Z_RANGE = 0.1
MOTOR_RANGES = torch.tensor([CHI_RANGE, OMEGA_RANGE, X_RANGE, Y_RANGE, Z_RANGE], device=device, requires_grad=False)

"""
Hyper Parameters:
NUM_INPUTS: Number of input positions of the neural network. 
            Every position consists of 6 states (chi, omega, x, y, z, intensity), 
            so the overall input size is 6 * NUM_INPUTS.
NUM_NETWORKS: Number of networks per generation.
SAMPLES_PER_GENERATION: Defines the amount of tests for each network. Every test gives one loss as output. 
                        All these losses are summed up and are divided by SAMPLES_PER_GENERATION, 
                        to get an average loss for each network.
NUM_BEST_MODELS: Defines the number of networks which are used as parents for the next generation.
"""
NUM_INPUTS = 100
NUM_NETWORKS = 20
SAMPLES_PER_GENERATION = 20
NUM_BEST_MODELS = 4

### Model:

In [10]:
class Linear_QNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear1 = nn.Linear(6 * int(input_size), 64)
        self.linear2 = nn.Linear(64, 5)

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x)).detach().to(device)
        x = torch.sigmoid(self.linear2(x)).detach().to(device)
        return x

    def save(self, file_name="model.pth"):
        model_folder_path = "./saves"
        if not os.path.exists(model_folder_path):
            os.makedirs(model_folder_path)

        file_name = os.path.join(model_folder_path, file_name)
        torch.save(self.state_dict(), file_name)

    def fill(self, pos1, pos2, agent):
        """

        :param pos1: first input state of neural network
        :param pos2: second input state of neural network
        :param agent: current agent
        :return: The denormalized motor positions, predicted by the network
        """
        x = torch.zeros(agent.num_inputs, 6, requires_grad=False, device=device)
        if pos1[-1] > pos2[-1]:
            max_intensity = pos1[-1]
            pos1[-1] = 1
            pos2[-1] = pos2[-1] / max_intensity
        else:
            max_intensity = pos2[-1]
            pos2[-1] = 1
            pos1[-1] = pos1[-1] / max_intensity

        x[0] = pos1
        x[1] = pos2
        for i in range(2, agent.num_inputs):
            normalized_action = self(x.flatten())
            true_action = agent.change_motor_pos(normalized_action, agent.motor_ranges, normalize=False)

            scene = Scene(*agent.props, agent.misalignment + true_action, agent.detection_angle)
            intensity = torch.tensor([np.sum(scene.detector.beam_profile.intensity).item()]).detach().to(device)
            result = torch.cat((normalized_action, intensity), dim=0).detach().to(device)

            if result[-1] > max_intensity:
                ratio = max_intensity / result[-1]
                x[:, -1] *= ratio
                max_intensity = result[-1]
                result[-1] = 1
            else:
                result[-1] = result[-1] / max_intensity
            x[i] = result

        return agent.change_motor_pos(self(x.flatten()), agent.motor_ranges, normalize=False)

### Agent:

In [11]:
class Agent(object):
    def __init__(self, inc_props, sample_props, det_props, detection_angle, motor_ranges, num_networks,
                 num_inputs, samples_per_generation, num_best_models, load):
        self.props = (inc_props, sample_props, det_props)
        self.detection_angle = detection_angle
        self.misalignment = self.change_motor_pos(self.random_values(5), motor_ranges, normalize=False)

        self.motor_ranges = motor_ranges
        self.spg = samples_per_generation
        self.num_inputs = num_inputs
        self.num_best_models = num_best_models
        self.num_networks = num_networks

        models = []
        if load:
            saved_model = Linear_QNet(num_inputs).cuda()
            saved_model.load_state_dict(torch.load(os.path.dirname(__file__) + "\\saves\\model.pth"))
            saved_model.eval()
            models.append(saved_model)
            for i in range(1, num_networks):
                model = self.mutate(saved_model)
                models.append(model)
        else:
            for i in range(num_networks):
                model = Linear_QNet(num_inputs).cuda()
                models.append(model)
        self.models = models

    def generate_start_positions(self):
        """

        :return: torch tensor with shape(samples_per_generation, 2, 6)
        """
        for i in range(self.spg):
            pos1, pos2 = self.random_values(), self.random_values()
            pos1_true = self.change_motor_pos(pos1, self.motor_ranges, normalize=False)
            pos2_true = self.change_motor_pos(pos2, self.motor_ranges, normalize=False)
            scene1 = Scene(*self.props, self.misalignment - pos1_true, self.detection_angle)
            scene2 = Scene(*self.props, self.misalignment - pos2_true, self.detection_angle)
            intensity1 = np.sum(scene1.detector.beam_profile.intensity)
            intensity2 = np.sum(scene2.detector.beam_profile.intensity)
            state1 = np.append(pos1.cpu().detach().numpy(), intensity1)
            state2 = np.append(pos2.cpu().detach().numpy(), intensity2)
            if i == 0:
                states = (np.stack([state1, state2]))
            else:
                states = np.concatenate((states, np.stack([state1, state2])), axis=0)
        return torch.from_numpy(states.reshape(self.spg, 2, 6)).detach().to(device)

    def evaluate_models(self, start_positions):
        """

        :param start_positions: all start positions for this generation
        :return: losses for each network
        """
        losses = []
        count = 0
        for model in self.models:
            count += 1
            print(count)
            loss = 0.0
            for i in range(len(start_positions)):
                pred = model.fill(start_positions[i][0], start_positions[i][1], self)
                loss += torch.norm(pred + self.misalignment).item()
            losses.append(loss / len(start_positions))
            print(loss / len(start_positions))
        return losses

    def generate_new_models(self, losses: list):
        """

        :param losses: losses for each network
        :return: Set the new generation of models
        """
        models_sorted = [x for _, x in sorted(zip(losses, self.models))]
        new_models = []
        modulo = (self.num_networks - self.num_best_models) % self.num_best_models
        for i in range(self.num_best_models):
            new_models.append(models_sorted[i])
            for j in range(int((self.num_networks - self.num_best_models) / self.num_best_models)):
                new_model = self.mutate(models_sorted[i])
                new_models.append(new_model)
            if i < modulo:
                new_model = self.mutate(models_sorted[i])
                new_models.append(new_model)
        self.models = new_models

    @staticmethod
    def mutate(model, mutation_power: float = 0.1):
        """

        :param model: neural network
        :param mutation_power: amount of mutation
        :return: mutated neural network
        """
        new_model = deepcopy(model)
        for param in new_model.parameters():
            param.data += mutation_power * torch.randn_like(param)
        return new_model

    @staticmethod
    def random_values(num_values: int = 5):
        """

        :param num_values: number of random values that should be generated
        :return: torch float tensor with random values
        """
        lst = []
        for i in range(num_values):
            lst.append(random.uniform(0, 1))
        return torch.FloatTensor(lst).detach().to(device)

    @staticmethod
    def change_motor_pos(current_positions, motor_maxima, normalize: bool):
        """

        :param current_positions: current motor positions
        :param motor_maxima: maximum values of motors
        :param normalize: True, if you want to normalize; False if you want to "denormalize"
        :return: List with normalized (or denormalized) positions
        """
        if len(current_positions) != len(motor_maxima):
            print("Number of positions and motor ranges do not fit")
        else:
            if normalize:
                normalized_positions = current_positions / (2 * motor_maxima) + 0.5
                return normalized_positions
            else:
                denormalized_positions = (current_positions - 0.5) * 2 * motor_maxima
                return denormalized_positions


### Main:

In [12]:
agent = Agent(INC_PROPS, SAMPLE_PROPS, DET_PROPS, DETECTION_ANGLE, MOTOR_RANGES, NUM_NETWORKS,
                  NUM_INPUTS, SAMPLES_PER_GENERATION, NUM_BEST_MODELS, load=False)

counter = 0
while True:
    start_positions = agent.generate_start_positions()
    counter += 1
    losses = agent.evaluate_models(start_positions)
    agent.generate_new_models(losses)
    if counter % 1 == 0:
        models_sorted = [x for _, x in sorted(zip(losses, agent.models))]
        best_model = models_sorted[0]
        best_model.save()
        with open(os.path.abspath('') + "/saves/plot_file.txt", "a") as file:
            file.write(str(counter) + " " + str(sum(losses) / NUM_NETWORKS) + "\n")

1
0.9526601046323776
2
0.8763418793678284
3
0.9018690913915635
4
1.0070947110652924
5
1.0025361776351929
6
0.9349523782730103
7
1.0788765728473664
8
0.7234202116727829
9
0.9679147720336914
10
1.2287577867507935
11
1.0404771625995637
12
1.0097496271133424
13
0.9562422871589661
14
1.0780797481536866
15
0.9802985489368439
16
0.8694892674684525
17
1.0951423525810242
18
1.0096909761428834
19
1.0422238767147065
20
0.898971489071846
1
0.7243351429700852
2
0.92617147564888
3
0.8549174636602401
4
0.7500039160251617
5
1.0044047683477402
6
0.8700114279985428
7
0.7800437957048416
8
0.7453481942415238
9
0.39677954763174056
10
1.1042831182479858
11
0.8757822036743164
12
0.8462899506092072
13
0.5399753779172898
14
0.9031365483999252
15
0.594827675819397
16
0.8990992426872253
17
0.7242441624403
18
0.799684152007103
19
0.6851334124803543
20
1.0123285174369812
1
0.39833128452301025
2
0.5558837801218033
3
0.2830298215150833
4
0.5583183228969574
5
0.42409847676754
6
0.5423537343740463
7
0.1973801821470260

0.0626968047581613
14
0.11364457160234451
15
0.23103514388203622
16
0.07308438587933778
17
0.20808097943663598
18
0.16359414383769036
19
0.1917961359024048
20
0.28473332673311236
1
0.05586633738130331
2
0.2865472674369812
3
0.23533082604408265
4
0.3052291825413704
5
0.6264828741550446
6
0.06598068699240685
7
0.19828531369566918
8
0.16952930875122546
9
0.25593761578202245
10
0.33072459101676943
11
0.06529961656779051
12
0.13826260305941104
13
0.3955480560660362
14
0.36974789947271347
15
0.23486088886857032
16
0.0730235617607832
17
0.27790590971708296
18
0.21965067088603973
19
0.12371244356036186
20
0.2155325159430504
1
0.05327881220728159
2
0.22428844794631003
3
0.21411790326237679
4
0.2927801042795181
5
0.19891063049435614
6
0.06265629548579454
7
0.21058378219604493
8
0.8634206175804138
9
0.03960426999256015
10
0.24820830002427102
11
0.062252024002373216
12
0.18959677442908288
13
0.1750185489654541
14
0.1398564137518406
15
0.6276015132665634
16
0.0695287637412548
17
0.3404205650091171


0.1662552572786808
3
0.3284809082746506
4
0.2735193282365799
5
0.2777308329939842
6
0.04489706847816706
7
0.26281088441610334
8
0.22740533351898193
9
0.23565196320414544
10
0.5785683512687683
11
0.04153421688824892
12
0.3996182531118393
13
0.3891813099384308
14
0.666716319322586
15
0.040565578173846005
16
0.05208325926214456
17
0.14582329392433166
18
0.07270740382373334
19
0.18729837387800216
20
0.09321932159364224
1
0.045461300667375325
2
0.24324822202324867
3
0.20618394613265992
4
0.3993389904499054
5
0.19730722680687904
6
0.045900650788098574
7
0.12009161673486232
8
0.11182463262230158
9
0.14726388528943063
10
0.18336827978491782
11
0.04300221670418978
12
0.14860888719558715
13
0.13477846086025239
14
0.09271906167268754
15
0.3995791032910347
16
0.04253988452255726
17
0.21526297703385353
18
0.09534356333315372
19
0.3386294782161713
20
0.14179090559482574
1
0.046814147289842366
2
0.21329142451286315
3
0.318017315864563
4
0.2329552687704563
5
0.250969710201025
6
0.039331191685050726
7


0.1626362506300211
11
0.0403512854129076
12
0.29613711684942245
13
0.24923658072948457
14
0.268923445045948
15
0.06925655826926232
16
0.04811018733307719
17
0.2690438009798527
18
0.1613130196928978
19
0.7159992009401321
20
0.22894566059112548
1
0.04017584323883057
2
0.247431980073452
3
0.14088194370269774
4
0.10120177902281284
5
0.13359727002680302
6
0.046702202688902614
7
0.06537412013858557
8
0.40870763957500456
9
0.17816897481679916
10
0.1268619041889906
11
0.03918099058791995
12
0.36890939623117447
13
0.1579827181994915
14
0.31154183596372603
15
0.33531027734279634
16
0.050317556411027906
17
0.7676680535078049
18
0.20372928231954573
19
0.24032378569245338
20
0.1307851027697325
1
0.04188425727188587
2
0.2979137755930424
3
0.29240173622965815
4
0.17687345147132874
5
0.1834699161350727
6
0.04260848043486476
7
0.11645126342773438
8
0.11626086309552193
9
0.13033047504723072
10
0.5685892760753631
11
0.03952961694449186
12
0.13847696408629417
13
0.1679859459400177
14
0.1836375489830971
15

0.33755702525377274
19
0.06739233732223511
20
0.19207044914364815
1
0.044915550388395786
2
0.08175062984228135
3
0.1795991189777851
4
0.10730997286736965
5
0.25363166630268097
6
0.042675743065774444
7
0.1943089984357357
8
0.11852804608643056
9
0.05910560842603445
10
0.23538932278752328
11
0.04092226102948189
12
0.24030377343297005
13
0.15372126996517183
14
0.11977026723325253
15
0.19857416525483132
16
0.04798285225406289
17
0.15385796539485455
18
0.23905449435114862
19
0.2121829390525818
20
0.1365337636321783
1
0.042866087146103383
2
0.6335615783929824
3
0.3104211285710335
4
0.2013545699417591
5
0.0884859561920166
6
0.03789464076980949
7
0.07758758924901485
8
0.3674596145749092
9
0.14854456409811972
10
0.17900078296661376
11
0.037388864811509846
12
0.34140183180570605
13
0.4611150845885277
14
0.2586659550666809
15
0.214904123544693
16
0.04318856634199619
17
0.21950165629386903
18
0.21910061612725257
19
0.20904821753501893
20
0.3714525416493416
1
0.040776355750858785
2
0.171327380090951

KeyboardInterrupt: 